In [9]:
#dataset link: https://drive.google.com/drive/folders/1n67sVTTzye4jtLfk8n-sa2fH2gTx5Ywt?usp=share_link

In [10]:
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
DATA_DIR = "./Dataset Korosi/"

In [11]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
    )

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    
)

train_generator = training_datagen.flow_from_directory(DATA_DIR, batch_size=32,class_mode='categorical',target_size=(150,150))   
val_generator = train_generator = training_datagen.flow_from_directory(DATA_DIR, subset='validation', batch_size=32,class_mode='categorical',target_size=(150,150))   

Found 304 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [12]:
resnet = tf.keras.applications.ResNet50(
    input_shape=(150,150,3),
    weights='imagenet',
    include_top=False,
)

In [13]:
model_resnet = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer( input_shape=(150,150,3)),
    resnet,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
])


In [14]:
import matplotlib.pyplot as plt

In [15]:
f = open("resnet_50.csv", "w")
f.write("epochs,loss,val_loss,acc,val_acc")
f.close()


In [16]:
Wsave = model_resnet.get_weights()
for times in range(10):
    epo = (times+1)*10
    model_resnet.set_weights(Wsave)
    model_resnet.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])
    history = model_resnet.fit(train_generator,validation_data=val_generator,epochs=epo, verbose = 0)

    f = open("resnet_50.csv", "a")
    f.write("\n"+str(epo)+","+
        str(history.history['loss'][-1])+","+
        str(history.history['val_loss'][-1])+","+
        str(history.history['acc'][-1])+","+
        str(history.history['val_acc'][-1])
    )
    f.close()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./resnet50/loss_'+str(epo)+'.png')
    plt.cla()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./resnet50/acc_'+str(epo)+'.png')
    
    plt.clf()

<Figure size 432x288 with 0 Axes>